# 데이터분석 순서  (KDD 분석 방법론)
* 데이터 세트 선택 : CSV, EXCEL, DB 에서 데이를 읽어옴
* 데이터 전처리 : 데이터타입, 결측값, 이상치 탐지, 데이터 분포 분석, 상관관계
* 데이터 변환 (특성추출) : 원본 데이터에서 새로운 데이터 생성, 삭제, 스케일링, 구간화
* 데이터 마이닝 (모델 만들기, 분석) : 분석에 적합한 알고리즘 선택, 모델 생성, 튜닝
* 결과 평가 : 테스트 데이터를 이용해서 데이터 마이닝으로 만든 모델의 성능 평가


# 데이터 전처리
* 데이터 타입 변환
* 

# info() 로 컬럼명, 결측치, 데이터 타입 파악

In [2]:
data.info()

NameError: name 'data' is not defined

In [65]:
import pandas as pd
data = pd.read_csv("./data/Titanic_train.csv")

# 4. 결측값 찾고, 비율 보고, 대치/삭제하기
* 결측값 비율 계산 : isna().sum()/len(데이터프레임) * 100
* 결측값 비율이 5% 미만: 행, 열을 제거, 분석에 크게 영향을 미치지 않음
* 결측값 비율이 5% ~ 30% : 결측값을 대체 (Implulation)
    * 수치형 데이터(숫자형, 나이, 가격): 평균(mean), 중앙값(median), 최빈값(mode)
    * 범주형 데이터(문자형/(숫자형), 선실등급, 탑승지: 최빈값(mode)으로 대체
* 결측값 비율이 30~50%: 컬럼의 중요도에 따라서 결측값을 대체 혹은 삭제
    * KNN(K-Nearest Neighbor, 최근접이웃) imputer, 회귀분석을 통해 결측값 대체
* 결측값 비율이 50% 이상: 해당 컬럼 삭제

## 결측 데이터의 종류
* MCAR(완전 무작위 결측): 다른 변수와 무관하게 생긴 결측(랜덤, 이유 없음)
    * 설문 조사중 일부 응답자가 무작위로 답변을 건너뜀
* MAR(무작위 결측): 다른 변수와는 관련 있지만 본인과는 무관
    * 나이가 많은 사람들이 소득란에 응답을 하지 않는 경우 (나이와 관련)
    * 직업에 따라서 학력란에 응답하지 않는 경우 (가정환경, 소득)
* NMAR(비무작위 결측): 변수 자기 자신과 직접 관련이 있는 경우
    * 소득이 낮은 사람이 자신의 소득을 숨기는 경우 (소득과 결측이 직접 관련)
    * 체중이 많이 나가는 경우 자신의 체중을 숨기는 경우
    * 만족도가 낮은 고객이 만족도 조사에 응하지 않는 경우

## 결측치 처리방법
* 1) 단순대치법 (simple impulation)
    * (1) 완전분석: 결측값이 있는 모든 행을 삭제하고 완전한 자료만으로 분석 (잘 안 씀)
    * 결측값을 삭제해도 모델을 만들기에 충분히 많은 데이터가 있는 경우
    * 결측값을 삭제한 후에 데이터에 편향이 없다는 전제가 있을 때
    * dropna(): 결측값이 있는 모든 행 삭제

In [66]:
data.dropna()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [6]:
183 / len(data) * 100

20.53872053872054

**(2) 평균 대치법: 결측치가 있는 컬럼에서 데이터의 평균을 구한 후 결측값을 대치**
* 평균을 이용하기 때문에 간편
* 데이터에 이상치가 있을 경우 평균을 이용할 수 없다.
* 데이터에 이상치가 있는 경우 중앙값이나 최빈값을 이용해야 함

In [7]:
a = np.array([24, 5, 10, 34, 20, 18, 28, 20])
b = np.array([24, 5, 10, 34, 20, 18, 28, 2000])

NameError: name 'np' is not defined

In [ ]:
a.mean()

In [8]:
b.mean()

NameError: name 'b' is not defined

In [9]:
a.median()

NameError: name 'a' is not defined

In [ ]:
sorted(a)

In [11]:
sorted(b)

NameError: name 'b' is not defined

Age 컬럼의 결측값을 평균 대치법으로 대치

In [15]:
data['Age'].describe()
#평균과 중앙값을 비교

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [18]:
age_na_idx = data[data['Age'].isna()].index

In [40]:
# 깊은 복사, 얕은 복사
data2 = data.copy()
data3 = data.copy()
data4 = data.copy()

In [24]:
data['Age'] = data['Age'].fillna(data['Age'].mean())

In [25]:
data.loc[age_na_idx]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q,29.699118
17,18,1,2,"Williams, Mr. Charles Eugene",male,29.699118,0,0,244373,13.0000,NaN,S,29.699118
19,20,1,3,"Masselmani, Mrs. Fatima",female,29.699118,0,0,2649,7.2250,NaN,C,29.699118
26,27,0,3,"Emir, Mr. Farred Chehab",male,29.699118,0,0,2631,7.2250,NaN,C,29.699118
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,29.699118,0,0,330959,7.8792,NaN,Q,29.699118
...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,29.699118,0,0,2629,7.2292,NaN,C,29.699118
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,29.699118,8,2,CA. 2343,69.5500,NaN,S,29.699118
868,869,0,3,"van Melkebeke, Mr. Philemon",male,29.699118,0,0,345777,9.5000,NaN,S,29.699118
878,879,0,3,"Laleff, Mr. Kristo",male,29.699118,0,0,349217,7.8958,NaN,S,29.699118


In [26]:
del data['age']

중앙값으로 대치

In [29]:
age_na_index2 = data2[data2['Age'].isna()].index

In [30]:
data2['Age'] = data2['Age'].fillna(data2['Age'].median())

In [31]:
data2.loc[age_na_index2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,28.0,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,28.0,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,28.0,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,28.0,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,28.0,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,28.0,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,28.0,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,28.0,0,0,349217,7.8958,NaN,S


최빈값으로 대치 .mode()

In [35]:
data3['Age'].mode()[0]

np.float64(24.0)

In [36]:
data['Age'] = data3['Age'].fillna(data3['Age'].mode())

In [37]:
age_na_idx3 = data3[data3['Age'].isna()].index
age_na_idx3

Index([  5,  17,  19,  26,  28,  29,  31,  32,  36,  42,
       ...
       832, 837, 839, 846, 849, 859, 863, 868, 878, 888],
      dtype='int64', length=177)

In [38]:
data3['Age'] = data3['Age'].fillna(data3['Age'].mode()[0])

In [34]:
data3.loc[age_na_index2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


# scikit-learn의 simple imputer를 이용한 대치

In [41]:
from sklearn.impute import SimpleImputer

In [44]:
imp_mean = SimpleImputer(strategy = 'mean')
data4['Age'] = imp_mean.fit_transform(data4[['Age']])[:,0]

In [49]:
data4['Age'].isna().sum()

np.int64(0)

In [73]:
na_idx = data3[data3['Age'].isna()].index

In [52]:
imp_median = SimpleImputer(strategy = 'median')
imp_median.fit_transform(data3[['Age']])[:, 0]

array([22.  , 38.  , 26.  , 35.  , 35.  , 28.  , 54.  ,  2.  , 27.  ,
       14.  ,  4.  , 58.  , 20.  , 39.  , 14.  , 55.  ,  2.  , 28.  ,
       31.  , 28.  , 35.  , 34.  , 15.  , 28.  ,  8.  , 38.  , 28.  ,
       19.  , 28.  , 28.  , 40.  , 28.  , 28.  , 66.  , 28.  , 42.  ,
       28.  , 21.  , 18.  , 14.  , 40.  , 27.  , 28.  ,  3.  , 19.  ,
       28.  , 28.  , 28.  , 28.  , 18.  ,  7.  , 21.  , 49.  , 29.  ,
       65.  , 28.  , 21.  , 28.5 ,  5.  , 11.  , 22.  , 38.  , 45.  ,
        4.  , 28.  , 28.  , 29.  , 19.  , 17.  , 26.  , 32.  , 16.  ,
       21.  , 26.  , 32.  , 25.  , 28.  , 28.  ,  0.83, 30.  , 22.  ,
       29.  , 28.  , 28.  , 17.  , 33.  , 16.  , 28.  , 23.  , 24.  ,
       29.  , 20.  , 46.  , 26.  , 59.  , 28.  , 71.  , 23.  , 34.  ,
       34.  , 28.  , 28.  , 21.  , 33.  , 37.  , 28.  , 21.  , 28.  ,
       38.  , 28.  , 47.  , 14.5 , 22.  , 20.  , 17.  , 21.  , 70.5 ,
       29.  , 24.  ,  2.  , 21.  , 28.  , 32.5 , 32.5 , 54.  , 12.  ,
       28.  , 24.  ,

In [53]:
data3.loc[na_idx]

PassengerId                           178
Survived                                0
Pclass                                  1
Name           Isham, Miss. Ann Elizabeth
Sex                                female
Age                                  50.0
SibSp                                   0
Parch                                   0
Ticket                           PC 17595
Fare                              28.7125
Cabin                                 C49
Embarked                                C
Name: 177, dtype: object

In [55]:
imp_mode = SimpleImputer(strategy = 'most_frequent')
data2['Age'] = imp_mode.fit_transform(data2[['Age']])[:, 0]

In [56]:
data2.loc[na_idx]

PassengerId                           178
Survived                                0
Pclass                                  1
Name           Isham, Miss. Ann Elizabeth
Sex                                female
Age                                  50.0
SibSp                                   0
Parch                                   0
Ticket                           PC 17595
Fare                              28.7125
Cabin                                 C49
Embarked                                C
Name: 177, dtype: object

In [59]:
imp_mode = SimpleImputer(strategy = 'most_frequent')
imp_mode.fit_transform(data2[['Age']])[:,0]

array([22.  , 38.  , 26.  , 35.  , 35.  , 24.  , 54.  ,  2.  , 27.  ,
       14.  ,  4.  , 58.  , 20.  , 39.  , 14.  , 55.  ,  2.  , 24.  ,
       31.  , 24.  , 35.  , 34.  , 15.  , 28.  ,  8.  , 38.  , 24.  ,
       19.  , 24.  , 24.  , 40.  , 24.  , 24.  , 66.  , 28.  , 42.  ,
       24.  , 21.  , 18.  , 14.  , 40.  , 27.  , 24.  ,  3.  , 19.  ,
       24.  , 24.  , 24.  , 24.  , 18.  ,  7.  , 21.  , 49.  , 29.  ,
       65.  , 24.  , 21.  , 28.5 ,  5.  , 11.  , 22.  , 38.  , 45.  ,
        4.  , 24.  , 24.  , 29.  , 19.  , 17.  , 26.  , 32.  , 16.  ,
       21.  , 26.  , 32.  , 25.  , 24.  , 24.  ,  0.83, 30.  , 22.  ,
       29.  , 24.  , 28.  , 17.  , 33.  , 16.  , 24.  , 23.  , 24.  ,
       29.  , 20.  , 46.  , 26.  , 59.  , 24.  , 71.  , 23.  , 34.  ,
       34.  , 28.  , 24.  , 21.  , 33.  , 37.  , 28.  , 21.  , 24.  ,
       38.  , 24.  , 47.  , 14.5 , 22.  , 20.  , 17.  , 21.  , 70.5 ,
       29.  , 24.  ,  2.  , 21.  , 24.  , 32.5 , 32.5 , 54.  , 12.  ,
       24.  , 24.  ,

In [76]:
data5 = data.copy

In [77]:
data5.isna()

AttributeError: 'function' object has no attribute 'isna'

* 결측값 비율이 30%-50%일 때 KNN 최근접이웃법

In [78]:
from sklearn.impute import KNNImputer

In [79]:
knn_imp = KNNImputer(n_neighbors=5)
knn_imp.fit_transform(data5[['Age']])[:, 0]

TypeError: 'method' object is not subscriptable

In [80]:
data5.loc[na_idx]

AttributeError: 'function' object has no attribute 'loc'

In [82]:
data4.loc[na_idx]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,29.699118,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,29.699118,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,29.699118,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,29.699118,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,29.699118,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,29.699118,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,29.699118,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,29.699118,0,0,349217,7.8958,NaN,S


# 깊은 복사, 얕은 복사

In [83]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [87]:
#data를 copy()를 이용해서 복사
data_copyed = data.copy()


# data를 다른 변수에 재할당
data_reassigned = data

In [88]:
data['Age'].isna().sum()

np.int64(177)

In [89]:
data_reassigned['Age'] = data_reassigned['Age'].fillna(28)

In [90]:
data_reassigned['Age'].isna().sum()

np.int64(0)

In [91]:
data['Age'].isna().sum()

np.int64(0)

In [92]:
data.loc[na_idx]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,28.0,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,28.0,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,28.0,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,28.0,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,28.0,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,28.0,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,28.0,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,28.0,0,0,349217,7.8958,NaN,S


# 실제 메모리상의 주소를 출력 id()

In [94]:
print("data의 메모리 주소: ", id(data))
print("data_reassigned의 메모리 주소: ", id(data_reassigned))
print("data_copy의 메모리 주소: ", id(data_copyed))

data의 메모리 주소:  2752215930240
data_reassigned의 메모리 주소:  2752215930240
data_copy의 메모리 주소:  2752215378960


In [96]:
data_copyed.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [97]:
new_df = data_copyed[['Survived', 'Pclass', 'Name']]

In [98]:
print("data_copyed: ", id(data_copyed))
print("new_df: ", id(new_df))

data_copyed:  2752215378960
new_df:  2752218206656


In [99]:
loc_result = data_copyed.loc[data_copyed['Age'] > 50, ['Name', 'Age']]
slice_result = data_copyed.loc[data_copyed['Age'] > 50][['Name', 'Age']]

In [100]:
loc_result

,Name,Age
6,"McCarthy, Mr. Timothy J",54.0
11,"Bonnell, Miss. Elizabeth",58.0
15,"Hewlett, Mrs. (Mary D Kingcome)",55.0
33,"Wheadon, Mr. Edward H",66.0
54,"Ostby, Mr. Engelhart Cornelius",65.0
...,...,...
820,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",52.0
829,"Stone, Mrs. George Nelson (Martha Evelyn)",62.0
851,"Svensson, Mr. Johan",74.0
857,"Daly, Mr. Peter Denis",51.0


In [101]:
slice_result

,Name,Age
6,"McCarthy, Mr. Timothy J",54.0
11,"Bonnell, Miss. Elizabeth",58.0
15,"Hewlett, Mrs. (Mary D Kingcome)",55.0
33,"Wheadon, Mr. Edward H",66.0
54,"Ostby, Mr. Engelhart Cornelius",65.0
...,...,...
820,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",52.0
829,"Stone, Mrs. George Nelson (Martha Evelyn)",62.0
851,"Svensson, Mr. Johan",74.0
857,"Daly, Mr. Peter Denis",51.0


In [103]:
print("data_copyed:", id(data_copyed))
print("data_result:", id(loc_result))
print("slice_result:", id(slice_result))

data_copyed: 2752215378960
data_result: 2752214227120
slice_result: 2752221747232


In [106]:
data.isna().sum() /len(data * 100)

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.000000
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64

In [107]:
data.drop("Cabin", axis=1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [108]:
data.drop("Cabin", axis=1, inplace=True)

In [109]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [112]:
data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

# 카테고리 변수의 경우 최빈값 .mode()를 이용해 결측값 처리

In [116]:
cabin_idx = data[data['Embarked'].isna()].index

In [117]:
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

In [119]:
data.loc[cabin_idx]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


In [122]:
data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [123]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked'],
      dtype='object')

In [126]:
data7 = data[['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked']]

#이중 대괄호는 깊은복사

In [127]:
print(id(data))
print(id(data7))

2752215930240
2752221740528
